In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [21]:
df = pd.read_csv('../data/Datos_de_calidad_del_agua_2020/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_subterraneas_2020.csv', encoding='ISO-8859-1')
df.head()

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,DLAGU6516,POZO R013 CAÑADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,2020,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,2020,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI


## Limpiando el dataset

In [22]:
columnas_remover = ['CLAVE', 'SITIO', 'ORGANISMO_DE_CUENCA', 'ESTADO', 'MUNICIPIO', 'ACUIFERO', 
'SUBTIPO', 'PERIODO', 'SDT_mg/L', 'CD_TOT_mg/L', 'HG_TOT_mg/L', 'PB_TOT_mg/L',
'COLI_FEC_NMP/100_mL', 'AS_TOT_mg/L', 'CR_TOT_mg/L', 'MN_TOT_mg/L', 'FE_TOT_mg/L'
]
df.drop(columnas_remover, axis=1, inplace=True)
# ALC_mg/L	CONDUCT_mS/cm	SDT_M_mg/L	DUR_mg/L	N_NO3_mg/L

In [23]:
df.head()

,LONGITUD,LATITUD,ALC_mg/L,CALIDAD_ALC,CONDUCT_mS/cm,CALIDAD_CONDUC,SDT_M_mg/L,CALIDAD_SDT_ra,CALIDAD_SDT_salin,FLUORUROS_mg/L,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,-102.02210,22.20887,229.990,Alta,940.0,Permisible para riego,603.6,Cultivos sensibles,Potable - Dulce,0.9766,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,-102.20075,21.99958,231.990,Alta,608.0,Buena para riego,445.4,Excelente para riego,Potable - Dulce,0.9298,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,-102.28801,22.36685,204.920,Alta,532.0,Buena para riego,342,Excelente para riego,Potable - Dulce,1.8045,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,-102.29449,22.18435,327.000,Alta,686.0,Buena para riego,478.6,Excelente para riego,Potable - Dulce,1.1229,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,-110.24480,23.45138,309.885,Alta,1841.0,Permisible para riego,1179,Cultivos con manejo especial,Ligeramente salobres,0.2343,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI


In [24]:
# REPLACE VALUES FOR THESE COLUMNS
df = df[df['SDT_M_mg/L'] != '<25']
df = df[df['DUR_mg/L'] != '<20']
df['N_NO3_mg/L'].replace('<0.02', 0, inplace=True)
df['FLUORUROS_mg/L'].replace('<0.2')
df.dropna(inplace=True)

df = df.astype({'SDT_M_mg/L':'float32', 'N_NO3_mg/L':'float32', 'DUR_mg/L':'float32'})

In [25]:
numerics = ['int64', 'int32', 'int64', 'float16', 'float32', 'float64']
df_numeric = df.select_dtypes(include=numerics)
df_numeric.head()

,LONGITUD,LATITUD,ALC_mg/L,CONDUCT_mS/cm,SDT_M_mg/L,DUR_mg/L,N_NO3_mg/L
2,-102.288010,22.366850,204.920,532.0,342.000000,120.719002,1.449803
4,-110.244800,23.451380,309.885,1841.0,1179.000000,476.987213,15.672251
5,-110.220670,23.464930,224.475,570.3,554.799988,201.878403,8.555740
7,-109.907306,22.890500,350.760,2253.3,1160.199951,269.171204,27.600998
9,-110.054722,23.824722,332.605,1703.0,1017.799988,559.021423,0.143061


In [26]:
df_numeric.isna().sum()

LONGITUD         0
LATITUD          0
ALC_mg/L         0
CONDUCT_mS/cm    0
SDT_M_mg/L       0
DUR_mg/L         0
N_NO3_mg/L       0
dtype: int64

In [27]:
df_numeric.describe()

,LONGITUD,LATITUD,ALC_mg/L,CONDUCT_mS/cm,SDT_M_mg/L,DUR_mg/L,N_NO3_mg/L
count,604.000000,604.000000,604.000000,604.000000,604.000000,604.000000,604.000000
mean,-101.737853,23.332116,255.888642,1483.175662,1221.766357,457.739136,5.586478
std,6.831866,3.733554,122.799179,1533.756739,3615.350830,434.088531,10.651263
min,-116.664250,15.064110,26.640000,188.000000,139.399994,21.940599,0.000000
25%,-104.916900,20.472502,182.730000,608.750000,432.400002,149.107502,0.706121
50%,-102.268905,23.006460,231.077500,1132.000000,778.000000,342.700806,2.645978
75%,-99.008530,25.521922,315.002500,1790.750000,1288.000000,613.043503,5.879826
max,-86.868880,32.677713,1650.000000,18577.000000,82170.000000,3810.692139,121.007812


In [28]:
df.to_csv('../data/data_subterraneos_clean.csv', encoding='ISO-8859-1', index=False)

In [29]:
target = ['SEMAFORO']
features = [v for v in df.columns.values.tolist() if v not in target]
target_df = df[target]
features_df = df[features]

In [37]:
not_numeric_cols = [key for key in dict(features_df.dtypes) if dict(features_df.dtypes)[key] not in numerics]
pd.get_dummies(features_df, prefix=not_numeric_cols)

,LONGITUD,LATITUD,ALC_mg/L,CONDUCT_mS/cm,SDT_M_mg/L,DUR_mg/L,N_NO3_mg/L,CALIDAD_ALC_Alta,CALIDAD_ALC_Baja,CALIDAD_ALC_Indeseable como FAAP,...,CUMPLE_CON_CR_NO,CUMPLE_CON_CR_SI,CUMPLE_CON_HG_NO,CUMPLE_CON_HG_SI,CUMPLE_CON_PB_NO,CUMPLE_CON_PB_SI,CUMPLE_CON_MN_NO,CUMPLE_CON_MN_SI,CUMPLE_CON_FE_NO,CUMPLE_CON_FE_SI
2,-102.288010,22.366850,204.920,532.0,342.000000,120.719002,1.449803,1,0,0,...,0,1,0,1,0,1,0,1,0,1
4,-110.244800,23.451380,309.885,1841.0,1179.000000,476.987213,15.672251,1,0,0,...,0,1,0,1,0,1,0,1,0,1
5,-110.220670,23.464930,224.475,570.3,554.799988,201.878403,8.555740,1,0,0,...,0,1,0,1,0,1,0,1,0,1
7,-109.907306,22.890500,350.760,2253.3,1160.199951,269.171204,27.600998,1,0,0,...,0,1,0,1,0,1,0,1,0,1
9,-110.054722,23.824722,332.605,1703.0,1017.799988,559.021423,0.143061,1,0,0,...,0,1,0,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,-99.434410,24.848210,410.625,1767.0,1011.400024,573.968018,25.168564,0,0,1,...,0,1,0,1,0,1,0,1,0,1
1062,-99.439140,24.974780,199.800,1622.0,1092.199951,679.271973,2.357554,1,0,0,...,0,1,0,1,0,1,0,1,0,1
1063,-99.541910,24.760360,231.045,2350.0,1545.800049,752.096008,14.615488,1,0,0,...,0,1,0,1,0,1,0,1,0,1
1064,-99.700990,24.782800,256.000,529.0,297.000000,273.000000,77.391998,1,0,0,...,0,1,0,1,0,1,0,1,0,1


In [30]:
new_data = pd.read_csv('../data/data_subterraneos_clean.csv')
new_data.dtypes

LONGITUD                float64
LATITUD                 float64
ALC_mg/L                float64
CALIDAD_ALC              object
CONDUCT_mS/cm           float64
CALIDAD_CONDUC           object
SDT_M_mg/L              float64
CALIDAD_SDT_ra           object
CALIDAD_SDT_salin        object
FLUORUROS_mg/L           object
CALIDAD_FLUO             object
DUR_mg/L                float64
CALIDAD_DUR              object
CALIDAD_COLI_FEC         object
N_NO3_mg/L              float64
CALIDAD_N_NO3            object
CALIDAD_AS               object
CALIDAD_CD               object
CALIDAD_CR               object
CALIDAD_HG               object
CALIDAD_PB               object
CALIDAD_MN               object
CALIDAD_FE               object
SEMAFORO                 object
CONTAMINANTES            object
CUMPLE_CON_ALC           object
CUMPLE_CON_COND          object
CUMPLE_CON_SDT_ra        object
CUMPLE_CON_SDT_salin     object
CUMPLE_CON_FLUO          object
CUMPLE_CON_DUR           object
CUMPLE_C